In [21]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms

import composer.functional as cf

import matplotlib.pyplot as plt
import os


data_path = 'dataset/'

In [22]:
# Transform
data_transforms = {
    'test':
        transforms.Compose([
            transforms.ToTensor(),
            # from ResNet requirement
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
}

In [23]:
# Data load and basic info
# Data load
image_datasets = {x: datasets.ImageFolder(os.path.join(data_path, x),data_transforms[x])
                  for x in ['test']}

# Into dataloaders
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=10, shuffle=True, num_workers=6)
              for x in ['test']}

# basic info
dataset_sizes = {x: len(image_datasets[x]) for x in ['test']}
class_names = image_datasets['test'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Info prints
print("Class Names:", class_names)

print("Train - Val:", dataset_sizes)
print("device available:", device)

Class Names: ['altar', 'apse', 'bell_tower', 'column', 'dome(inner)', 'dome(outer)', 'flying_buttress', 'gargoyle', 'stained_glass', 'vault']
Train - Val: {'test': 1404}
device available: cpu


In [24]:
# Reconstruct Model
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))
model_ft = model_ft.to(device)

# Composer speed up
model_ft = cf.apply_blurpool(model_ft)
model_ft = cf.apply_squeeze_excite(model_ft)

# load weights
model_ft.load_state_dict(torch.load('model50.pth'))

criterion = nn.CrossEntropyLoss()

In [27]:
# Fixed Feature Extractor
model_conv = torchvision.models.resnet50(pretrained=True)
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, len(class_names))
model_conv = model_conv.to(device)

# Composer speed up
model_conv = cf.apply_blurpool(model_conv)
model_conv = cf.apply_squeeze_excite(model_conv)

# load weights
model_conv.load_state_dict(torch.load('modelff.pth'))

<All keys matched successfully>

In [25]:
def test_model(model, criterion, test_dir):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders[test_dir]):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
        loss = running_loss / dataset_sizes[test_dir]
        acc = running_corrects.double() / dataset_sizes[test_dir]
        
        print(f'{test_dir} Loss: {loss:.4f} Acc: {acc:.4f}')

In [26]:
test_model(model_ft, criterion, 'test')

test Loss: 0.2550 Acc: 0.9323


In [28]:
test_model(model_conv, criterion, 'test')

test Loss: 0.6559 Acc: 0.7756
